<a href="https://colab.research.google.com/github/mbrudd/LLMs/blob/main/.ipynb_checkpoints/Attention%20mechanisms%2C%20part%201-checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import tiktoken

In [ ]:
raw_text = "This is my dog Hazel"

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
enc_text = tokenizer.encode( raw_text )
print(enc_text)

[1212, 318, 616, 3290, 42805]


In [ ]:
vocab_size = 4
output_dim = 8
inputs = torch.nn.Embedding( vocab_size, output_dim )
print( inputs.weight )

Parameter containing:
tensor([[ 0.6078,  0.5842,  0.8049, -0.5765,  2.3375,  1.0910,  0.1613,  0.1589],
        [ 0.6926,  0.4180, -0.9213, -0.7376,  2.2582,  0.5391, -0.1460,  0.1086],
        [-1.1629,  0.3454, -0.2034, -0.4822, -0.6479,  0.8667, -0.1860, -0.7852],
        [-0.8467,  1.8198, -0.1719, -0.1674, -0.0121, -0.7557, -0.8369,  0.9339]],
       requires_grad=True)


In [ ]:
inputs = inputs.weight.data
inputs

tensor([[ 0.6078,  0.5842,  0.8049, -0.5765,  2.3375,  1.0910,  0.1613,  0.1589],
        [ 0.6926,  0.4180, -0.9213, -0.7376,  2.2582,  0.5391, -0.1460,  0.1086],
        [-1.1629,  0.3454, -0.2034, -0.4822, -0.6479,  0.8667, -0.1860, -0.7852],
        [-0.8467,  1.8198, -0.1719, -0.1674, -0.0121, -0.7557, -0.8369,  0.9339]])

In [ ]:
inputs.shape

torch.Size([4, 8])

In [ ]:
for row in inputs:
    print(row.tolist())

[0.6078398823738098, 0.5842005610466003, 0.8049112558364868, -0.5765498280525208, 2.337498903274536, 1.0910053253173828, 0.16126438975334167, 0.15888425707817078]
[0.6926334500312805, 0.41795268654823303, -0.921288251876831, -0.7376049160957336, 2.2582156658172607, 0.5391291975975037, -0.14602220058441162, 0.10855613648891449]
[-1.1629329919815063, 0.3454475998878479, -0.20340225100517273, -0.4821770191192627, -0.647865891456604, 0.866673469543457, -0.18599539995193481, -0.7852029204368591]
[-0.8467072248458862, 1.8198200464248657, -0.17190590500831604, -0.1674225926399231, -0.0120980991050601, -0.7557389140129089, -0.8369002342224121, 0.9339028596878052]


In [ ]:
x = torch.Tensor([1.1, 2.3])
y = torch.Tensor([3.4,-2.1])


In [ ]:
1.1*3.4 + 2.3*(-2.1)

-1.0899999999999999

In [ ]:
torch.dot( x, y)

tensor(-1.0900)

In [ ]:
query = inputs[2]
print(query)

tensor([-1.1629,  0.3454, -0.2034, -0.4822, -0.6479,  0.8667, -0.1860, -0.7852])


In [ ]:
for i in range( len( inputs )):
    print( torch.dot( query, inputs[i] ) )

tensor(-1.1144)
tensor(-1.1719)
tensor(3.5676)
tensor(0.5042)


In [ ]:
attention_scores_2 = torch.zeros( len(inputs) )
for i in range( len( inputs )):
    attention_scores_2[i] = torch.dot( query, inputs[i] )
print( attention_scores_2)

tensor([-1.1144, -1.1719,  3.5676,  0.5042])


In [ ]:
# normalize the attention scores using the softmax function:
# def softmax(x):
#     torch.exp(x) / torch.exp(x).sum()

In [ ]:
attention_weights_2 = torch.softmax( attention_scores_2, dim = 0 )
attention_weights_2

tensor([0.0087, 0.0082, 0.9392, 0.0439])

In [ ]:
attention_weights_2.sum()

tensor(1.)

In [ ]:
context_vector_2 = torch.zeros( query.shape )
for i in range( len( attention_weights_2 ) ):
    context_vector_2 += attention_weights_2[i]*inputs[i]
context_vector_2

tensor([-1.1184,  0.4128, -0.1991, -0.4713, -0.5701,  0.7947, -0.2112, -0.6942])

In [ ]:
inputs

tensor([[ 0.6078,  0.5842,  0.8049, -0.5765,  2.3375,  1.0910,  0.1613,  0.1589],
        [ 0.6926,  0.4180, -0.9213, -0.7376,  2.2582,  0.5391, -0.1460,  0.1086],
        [-1.1629,  0.3454, -0.2034, -0.4822, -0.6479,  0.8667, -0.1860, -0.7852],
        [-0.8467,  1.8198, -0.1719, -0.1674, -0.0121, -0.7557, -0.8369,  0.9339]])

In [ ]:
inputs.T

tensor([[ 0.6078,  0.6926, -1.1629, -0.8467],
        [ 0.5842,  0.4180,  0.3454,  1.8198],
        [ 0.8049, -0.9213, -0.2034, -0.1719],
        [-0.5765, -0.7376, -0.4822, -0.1674],
        [ 2.3375,  2.2582, -0.6479, -0.0121],
        [ 1.0910,  0.5391,  0.8667, -0.7557],
        [ 0.1613, -0.1460, -0.1860, -0.8369],
        [ 0.1589,  0.1086, -0.7852,  0.9339]])

In [ ]:
# get all of the attention scores via a matrix multiplication:
attention_scores = inputs @ inputs.T
attention_scores

tensor([[ 8.3965,  6.2094, -1.1144, -0.3327],
        [ 6.2094,  7.4706, -1.1719,  0.2448],
        [-1.1144, -1.1719,  3.5676,  0.5042],
        [-0.3327,  0.2448,  0.5042,  6.2301]])

In [ ]:
attention_weights = torch.softmax( attention_scores, dim = -1 )
attention_weights

tensor([[8.9890e-01, 1.0089e-01, 6.6556e-05, 1.4543e-04],
        [2.2061e-01, 7.7868e-01, 1.3740e-04, 5.6659e-04],
        [8.6976e-03, 8.2114e-03, 9.3920e-01, 4.3888e-02],
        [1.4018e-03, 2.4976e-03, 3.2372e-03, 9.9286e-01]])

In [ ]:
attention_weights[0].sum()

tensor(1.)

In [ ]:
context_vectors = attention_weights @ inputs
context_vectors

tensor([[ 0.6161,  0.5676,  0.6305, -0.5927,  2.3290,  1.0350,  0.1301,  0.1539],
        [ 0.6728,  0.4554, -0.5399, -0.7017,  2.2740,  0.6602, -0.0786,  0.1200],
        [-1.1184,  0.4128, -0.1991, -0.4713, -0.5701,  0.7947, -0.2112, -0.6942],
        [-0.8418,  1.8098, -0.1725, -0.1704, -0.0052, -0.7447, -0.8317,  0.9252]])